# 根據 CartPole 的遊戲，來實作 Policy Gradient
---

In [ ]:
%matplotlib inline
import gym
import matplotlib.pyplot as plt
from  RLAgent_PolicyGradient import PolicyGradient

## 建造遊戲環境
---
![](imgs/input_output.png "環境參數圖")
[原始連結](https://github.com/openai/gym/wiki/CartPole-v0)

In [ ]:
env = gym.make("CartPole-v0")                # 立竿子的遊戲
env = env.unwrapped                          # 遊戲設定，好像不要讓參數有限制

print(env.action_space)
print(env.observation_space)
print(env.observation_space.high)            # 最大值
print(env.observation_space.low)             # 最小值

## 測試遊戲
---
Example:  
<pre>array([ 0.10472821,  3.1417834 ,  1.96147299,  5.57941823]), 0.0, True, {}</pre>  
前面四個代表 Observation 的值  
再來是 Reward (這裡注意，在還可以救起來之前，Reward 都是 1，且 IsDone 為 False)  
Info 好像沒有用  

而** Action 是一個 Int => 0 or 1 !!**

In [ ]:
# 重製 & render
# print(env.reset())
# env.render()

In [ ]:
# env.render()
# print(env.step(1))

In [ ]:
# env.render(close=True)